## Deliverable 2. Create a Customer Travel Destinations Map.

In [27]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [28]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,2022-04-09 00:46:49,-33.5906,26.8910,56.17,95,88,9.71,overcast clouds
1,1,Klyuchi,RU,2022-04-09 00:46:50,52.2667,79.1667,33.96,70,0,6.62,clear sky
2,2,Hobart,AU,2022-04-09 00:46:51,-42.8794,147.3294,69.22,61,75,12.66,broken clouds
3,3,Cherskiy,RU,2022-04-09 00:46:52,68.7500,161.3000,-6.57,84,32,5.35,scattered clouds
4,4,Vaini,TO,2022-04-09 00:46:53,-21.2000,-175.2000,87.96,70,40,4.61,scattered clouds


In [29]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [30]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Vaini,TO,2022-04-09 00:46:53,-21.2000,-175.2000,87.96,70,40,4.61,scattered clouds
5,5,Hithadhoo,MV,2022-04-09 00:46:54,-0.6000,73.0833,83.12,68,97,9.35,overcast clouds
6,6,Cayenne,GF,2022-04-09 00:46:55,4.9333,-52.3333,81.09,1,20,7.00,few clouds
11,11,Puerto Ayora,EC,2022-04-09 00:46:56,-0.7393,-90.3518,77.81,86,41,5.84,scattered clouds
14,14,Dingle,PH,2022-04-09 00:46:57,10.9995,122.6711,79.00,80,100,4.09,overcast clouds
17,17,Avarua,CK,2022-04-09 00:46:59,-21.2078,-159.7750,86.05,74,100,11.50,overcast clouds
24,24,Bathsheba,BB,2022-04-09 00:47:01,13.2167,-59.5167,77.61,88,75,16.11,broken clouds
25,25,Acarau,BR,2022-04-09 00:45:35,-2.8856,-40.1200,77.47,88,94,9.04,overcast clouds
33,33,Butaritari,KI,2022-04-09 00:47:08,3.0707,172.7902,80.22,82,100,13.47,light rain
35,35,Buala,SB,2022-04-09 00:47:08,-8.1450,159.5921,85.05,70,46,9.89,light rain


In [25]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                205
City                   205
Country                205
Date                   205
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [26]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.count()

City_ID                205
City                   205
Country                205
Date                   205
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Vaini,TO,87.96,scattered clouds,-21.2000,-175.2000,
5,Hithadhoo,MV,83.12,overcast clouds,-0.6000,73.0833,
6,Cayenne,GF,81.09,few clouds,4.9333,-52.3333,
11,Puerto Ayora,EC,77.81,scattered clouds,-0.7393,-90.3518,
14,Dingle,PH,79.00,overcast clouds,10.9995,122.6711,
17,Avarua,CK,86.05,overcast clouds,-21.2078,-159.7750,
24,Bathsheba,BB,77.61,broken clouds,13.2167,-59.5167,
25,Acarau,BR,77.47,overcast clouds,-2.8856,-40.1200,
33,Butaritari,KI,80.22,light rain,3.0707,172.7902,
35,Buala,SB,85.05,light rain,-8.1450,159.5921,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found ...skipping.")
       
        

Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.


In [10]:
#Adding dependency
import numpy as np


In [14]:
# 7. Drop the rows where there is no Hotel Name.

#Replace empty string values with NaN panda objects in the Hotel Name Column
hotel_df['Hotel Name'].replace('',np.nan,inplace=True)

#Drop the rows where there are NaN values in the Hotel Name column
hotel_df.dropna(subset=['Hotel Name'], inplace=True)


In [15]:
#Create a new dataframe to store the location,weather data and nearest hotel.
clean_hotel_df = hotel_df
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Vaini,TO,87.96,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Hithadhoo,MV,83.12,overcast clouds,-0.6000,73.0833,Scoop Guest House
6,Cayenne,GF,81.09,few clouds,4.9333,-52.3333,Hôtel Le Dronmi
11,Puerto Ayora,EC,77.81,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
14,Dingle,PH,79.00,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
17,Avarua,CK,86.05,overcast clouds,-21.2078,-159.7750,Paradise Inn
24,Bathsheba,BB,77.61,broken clouds,13.2167,-59.5167,Atlantis Hotel
25,Acarau,BR,77.47,overcast clouds,-2.8856,-40.1200,castelo pizzaria
33,Butaritari,KI,80.22,light rain,3.0707,172.7902,Isles Sunset Lodge
35,Buala,SB,85.05,light rain,-8.1450,159.5921,Maringe Lagoon Lodge


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 

max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))